<a href="https://colab.research.google.com/github/elephanti/NLPProject2024/blob/main/Train_and_Test_with_augmented_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/elephanti/NLPProject2024.git

Cloning into 'NLPProject2024'...
remote: Enumerating objects: 988, done.
remote: Counting objects: 100% (988/988), done.
remote: Compressing objects: 100% (812/812), done.
remote: Total 988 (delta 282), reused 804 (delta 169), pack-reused 0
Receiving objects: 100% (988/988), 32.44 MiB | 15.47 MiB/s, done.
Resolving deltas: 100% (282/282), done.


In [7]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip -d NLPProject2024/classifiers/embeddings

--2024-08-08 00:16:00--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-08-08 00:16:00--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-08-08 00:16:01--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [2]:
!pip install tensorflow==2.15.1
!pip install ktrain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard
    Found existing installatio

In [39]:
!pip install transformers==4.37.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


def load_data(full_train_path, generated_train_path=None, test_path=None):
    print(f"\tFull train path: {full_train_path}")
    print(f"\tGenerated train path: {generated_train_path}")
    print(f"\tTest path: {test_path}")
    train_df = pd.read_csv(full_train_path)

    if generated_train_path:
        train_df = pd.concat([train_df, pd.read_csv(generated_train_path)], ignore_index=True)

    train_text = train_df['text'].tolist()
    train_label = train_df['label']

    le = LabelEncoder()
    y_train = le.fit_transform(train_label)

    if test_path:
        test_df = pd.read_csv(test_path)
        test_text = test_df['text'].tolist()
        test_label = test_df['label']
        y_test = le.transform(test_label)
        return train_text, y_train, test_text, y_test, le

    return train_text, y_train, le

In [9]:
import pandas as pd
import glob
import os
import joblib
from NLPProject2024.classifiers.bert_ktrain import DistilBERT
from NLPProject2024.classifiers.lstm_glove import LSTMGlove
from NLPProject2024.classifiers.svm_glove import SVMGlove
from NLPProject2024.classifiers.svm_tfidf import SVMTFIDF
import time

class ModelTrainer:
    def __init__(self, classifier_name, glove_file, num_labels):
        self.classifier = None
        self.classifier_name = classifier_name
        self.glove_file = glove_file
        self.num_labels = num_labels

    def train_classifier(self, X_train, y_train, model_path, X_test=None, y_test=None):
        print(f"Training {self.classifier_name}...")
        start_time = time.time()
        if self.classifier_name == 'lstm_glove':
            self.classifier = LSTMGlove
            clf_instance = self.classifier(self.glove_file, self.num_labels)
        elif self.classifier_name == 'svm_glove':
            self.classifier = SVMGlove
            clf_instance = self.classifier(self.glove_file)
        elif self.classifier_name == 'bert_ktrain':
            self.classifier = DistilBERT
            clf_instance = self.classifier(epochs=10)
        else:
            self.classifier = SVMTFIDF
            clf_instance = self.classifier()

        clf_instance.train(X_train, y_train)

        accuracy = clf_instance.evaluate(X_test, y_test)
        if self.classifier_name in ['lstm_glove', 'bert_ktrain']:
            results = {"classifier": self.classifier_name, "test_accuracy": round(accuracy.item(), 4)}
        else:
            results = {"classifier": self.classifier_name, "test_accuracy": round(accuracy, 4)}

        clf_instance.save(f'{model_path}/{self.classifier_name}')
        end_time = time.time()
        results["training_time"] = round(end_time - start_time, 2)

        return results


In [3]:
df_result = pd.DataFrame(columns=['dataset', 'model', 'test_accuracy'])
glove_file = 'NLPProject2024/classifiers/embeddings/glove.6B.100d.txt'

classifiers = {
    'svm_tfidf': SVMTFIDF,
    'svm_glove': SVMGlove,
    'lstm_glove': LSTMGlove,
    'bert_ktrain': DistilBERT
}

lambadas = ['Lambada', 'Lambada+', 'Lambada+Instruct']
llms = ['Llama3', 'Mistral']
dataset_names = ['ATIS', 'TREC']

In [27]:
full_df_result = pd.DataFrame(columns=['dataset', 'model', 'test_accuracy'])

# Train on full training data
for name in dataset_names:
    X_train, y_train, X_test, y_test, le = load_data(f'NLPProject2024/datasets/{name}/{name.lower()}.train.csv',
                                                      test_path=f'NLPProject2024/datasets/{name}/{name.lower()}.test.csv')
    model_path = f'final_results/models/{name.lower()}_train'
    os.makedirs(model_path, exist_ok=True)

    joblib.dump(le, f'{model_path}/label_encoder.pkl')

    for classifier in classifiers:
        print(f"Training {name} with {classifier}...")
        trainer = ModelTrainer(classifier, glove_file, len(le.classes_))
        train_results = trainer.train_classifier(X_train, y_train, model_path, X_test=X_test, y_test=y_test)

        train_results["dataset"] = f'{name.lower()}_full'
        full_df_result = pd.concat([full_df_result, pd.DataFrame([train_results])], ignore_index=True)
        print(f"Finished training {name} with {classifier}")
        print(train_results)
        print("-" * 50)

	Full train path: NLPProject2024/datasets/ATIS/atis.train.csv
	Generated train path: None
	Test path: NLPProject2024/datasets/ATIS/atis.test.csv
Training ATIS with svm_tfidf...
Training svm_tfidf...
Model saved.
Finished training ATIS with svm_tfidf
{'classifier': 'svm_tfidf', 'test_accuracy': 0.9588, 'training_time': 6.59, 'dataset': 'atis_full'}
--------------------------------------------------
Training ATIS with svm_glove...
Training svm_glove...
Model saved.
Finished training ATIS with svm_glove
{'classifier': 'svm_glove', 'test_accuracy': 0.928, 'training_time': 11.3, 'dataset': 'atis_full'}
--------------------------------------------------
Training ATIS with lstm_glove...
Training lstm_glove...
Epoch 1/20, Loss: 0.8709, Accuracy: 0.7875
Epoch 2/20, Loss: 0.3532, Accuracy: 0.9121
Epoch 3/20, Loss: 0.2231, Accuracy: 0.9464
Epoch 4/20, Loss: 0.1749, Accuracy: 0.9545
Epoch 5/20, Loss: 0.1286, Accuracy: 0.9653
Epoch 6/20, Loss: 0.1094, Accuracy: 0.9726
Epoch 7/20, Loss: 0.0832, Accu

In [11]:
subsets_df_result = pd.DataFrame(columns=['dataset', 'classifier', 'test_accuracy', 'model', 'subset_size', 'method'])

# Train on the subsets
for name in dataset_names:
    for subset_size in [5]:
        subset_path = f'NLPProject2024/datasets/{name}/sampled_subsets/ver1/{name.lower()}_{subset_size}_subset.csv'

        # subset only training
        X_train, y_train, X_test, y_test, le = load_data(subset_path,
                                                        test_path=f'NLPProject2024/datasets/{name}/{name.lower()}.test.csv')
        model_path = f'final_results/models/{name.lower()}_{subset_size}_subset_train'
        os.makedirs(model_path, exist_ok=True)

        joblib.dump(le, f'{model_path}/label_encoder.pkl')

        for classifier in classifiers:
            print(f"Training {name} with {classifier} and subset size {subset_size}...")
            trainer = ModelTrainer(classifier, glove_file, len(le.classes_))
            train_results = trainer.train_classifier(X_train, y_train, model_path, X_test=X_test, y_test=y_test)

            train_results["dataset"] = f'{name.lower()}_{subset_size}_subset_train'
            train_results["method"] = None
            train_results["model"] = None
            train_results["subset_size"] = subset_size
            subsets_df_result = pd.concat([subsets_df_result, pd.DataFrame([train_results])], ignore_index=True)
            print(f"Finished training {name} with {classifier} and subset size {subset_size}")
            print(train_results)
            print("-" * 50)

	Full train path: NLPProject2024/datasets/ATIS/sampled_subsets/ver1/atis_5_subset.csv
	Generated train path: None
	Test path: NLPProject2024/datasets/ATIS/atis.test.csv
Training ATIS with svm_tfidf and subset size 5...
Training svm_tfidf...
Model saved.
Finished training ATIS with svm_tfidf and subset size 5
{'classifier': 'svm_tfidf', 'test_accuracy': 0.6055, 'training_time': 0.04, 'dataset': 'atis_5_subset_train', 'method': None, 'model': None, 'subset_size': 5}
--------------------------------------------------
Training ATIS with svm_glove and subset size 5...
Training svm_glove...
Model saved.
Finished training ATIS with svm_glove and subset size 5
{'classifier': 'svm_glove', 'test_accuracy': 0.5815, 'training_time': 8.4, 'dataset': 'atis_5_subset_train', 'method': None, 'model': None, 'subset_size': 5}
--------------------------------------------------
Training ATIS with lstm_glove and subset size 5...
Training lstm_glove...
Epoch 1/20, Loss: 3.2915, Accuracy: 0.0732
Epoch 2/20, L

/usr/local/lib/python3.10/dist-packages/ktrain/text/preprocessor.py:382: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn(


preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 18
	99percentile : 23


Is Multi-Label? False


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
9/9 [==============================] - 8s 54ms/step - loss: 2.8924 - accuracy: 0.0732
Epoch 2/10
9/9 [==============================] - 0s 44ms/step - loss: 2.8685 - accuracy: 0.1220
Epoch 3/10
9/9 [==============================] - 0s 45ms/step - loss: 2.7724 - accuracy: 0.2927
Epoch 4/10
9/9 [==============================] - 0s 46ms/step - loss: 2.5052 - accuracy: 0.5244
Epoch 5/10
9/9 [==============================] - 0s 45ms/step - loss: 2.0888 - accuracy: 0.8537
Epoch 6/10
9/9 [==============================] - 0s 46ms/step - loss: 1.5762 - accuracy: 0.9390
Epoch 7/10
9/9 [==============================] - 0s 46ms/step - loss: 1.1814 - accuracy: 0.9512
Epoch 8/10
9/9 [==============================] - 0s 46ms/step - loss: 0.9037 - accuracy: 0.9756
Epoch 9/10
9/9 [==============================] - 0s 44ms/step - loss: 0.7652 - accuracy: 0.9756
Epoch 10/10
9/9 [=========================

/usr/local/lib/python3.10/dist-packages/ktrain/text/preprocessor.py:382: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn(


preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 21


Is Multi-Label? False


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
25/25 [==============================] - 9s 46ms/step - loss: 3.9153 - accuracy: 0.0160
Epoch 2/10
25/25 [==============================] - 1s 46ms/step - loss: 3.8856 - accuracy: 0.0560
Epoch 3/10
25/25 [==============================] - 1s 46ms/step - loss: 3.6981 - accuracy: 0.2800
Epoch 4/10
25/25 [==============================] - 1s 46ms/step - loss: 3.2735 - accuracy: 0.7560
Epoch 5/10
25/25 [==============================] - 1s 46ms/step - loss: 2.7732 - accuracy: 0.8480
Epoch 6/10
25/25 [==============================] - 1s 46ms/step - loss: 2.2046 - accuracy: 0.9640
Epoch 7/10
25/25 [==============================] - 1s 47ms/step - loss: 1.7152 - accuracy: 0.9880
Epoch 8/10
25/25 [==============================] - 1s 47ms/step - loss: 1.4090 - accuracy: 0.9920
Epoch 9/10
25/25 [==============================] - 1s 47ms/step - loss: 1.2229 - accuracy: 1.0000
Epoch 10/10
25/25 [=====

In [12]:
augmented_df_result = pd.DataFrame(columns=['dataset', 'classifier', 'test_accuracy', 'model', 'subset_size', 'method'])

# Train on subset + matching augmented data
for name in ['ATIS']:
    for subset_size in [5]:
        subset_path = f'NLPProject2024/datasets/{name}/sampled_subsets/ver1/{name.lower()}_{subset_size}_subset.csv'

        for lambada in lambadas:
            for llm in llms:
                if lambada == 'Lambada+Instruct' and llm == 'Mistral':
                    continue

                for classifier in classifiers:
                    directory_path = f'NLPProject2024/filtered_datasets/{lambada}/{llm}/{name}/{classifier}'

                    data_files = glob.glob(os.path.join(directory_path, '*.csv'))

                    for file in data_files:
                      # Hack to train with only matching augmented data
                      if f"{name.lower()}_{subset_size}_" not in file.lower():
                        continue
                      try:
                        dataset_file_name = file.replace(f'{directory_path}', "").replace('.csv', "").replace('/', "")
                        model_path = f'final_results/models/{dataset_file_name}'
                        os.makedirs(model_path, exist_ok=True)
                        print(f"Training {name} with {classifier} on {dataset_file_name}...")
                        X_train, y_train, X_test, y_test, le = load_data(subset_path,
                                                                        generated_train_path=file,
                                                                        test_path=f'NLPProject2024/datasets/{name}/{name.lower()}.test.csv')
                        joblib.dump(le, f'{model_path}/label_encoder.pkl')

                        trainer = ModelTrainer(classifier, glove_file, len(le.classes_))
                        train_results = trainer.train_classifier(X_train, y_train, model_path, X_test=X_test, y_test=y_test)

                        train_results["dataset"] = dataset_file_name
                        train_results["method"] = lambada
                        train_results["model"] = llm
                        train_results["subset_size"] = subset_size
                        augmented_df_result = pd.concat([augmented_df_result, pd.DataFrame([train_results])], ignore_index=True)
                        print(f"Finished training {name} with {classifier} on {dataset_file_name}")
                        print(train_results)
                        print("-" * 50)
                      except Exception as e:
                        print(f"Error training {name} with {classifier} on {dataset_file_name}: {e}")
                        continue

Training ATIS with svm_tfidf on Llama3_8B_ATIS_5_augmented_data_svm_tfidf_filtered...
	Full train path: NLPProject2024/datasets/ATIS/sampled_subsets/ver1/atis_5_subset.csv
	Generated train path: NLPProject2024/filtered_datasets/Lambada/Llama3/ATIS/svm_tfidf/Llama3_8B_ATIS_5_augmented_data_svm_tfidf_filtered.csv
	Test path: NLPProject2024/datasets/ATIS/atis.test.csv
Training svm_tfidf...
Model saved.
Finished training ATIS with svm_tfidf on Llama3_8B_ATIS_5_augmented_data_svm_tfidf_filtered
{'classifier': 'svm_tfidf', 'test_accuracy': 0.693, 'training_time': 0.12, 'dataset': 'Llama3_8B_ATIS_5_augmented_data_svm_tfidf_filtered', 'method': 'Lambada', 'model': 'Llama3', 'subset_size': 5}
--------------------------------------------------
Training ATIS with svm_glove on Llama3_8B_ATIS_5_augmented_data_svm_glove_filtered...
	Full train path: NLPProject2024/datasets/ATIS/sampled_subsets/ver1/atis_5_subset.csv
	Generated train path: NLPProject2024/filtered_datasets/Lambada/Llama3/ATIS/svm_glov

/usr/local/lib/python3.10/dist-packages/ktrain/text/preprocessor.py:382: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn(


preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 20


Is Multi-Label? False


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
23/23 [==============================] - 9s 48ms/step - loss: 2.8842 - accuracy: 0.0588
Epoch 2/10
23/23 [==============================] - 1s 46ms/step - loss: 2.7074 - accuracy: 0.3620
Epoch 3/10
23/23 [==============================] - 1s 45ms/step - loss: 2.0136 - accuracy: 0.8009
Epoch 4/10
23/23 [==============================] - 1s 45ms/step - loss: 1.1515 - accuracy: 0.9276
Epoch 5/10
23/23 [==============================] - 1s 46ms/step - loss: 0.5628 - accuracy: 0.9774
Epoch 6/10
23/23 [==============================] - 1s 45ms/step - loss: 0.3059 - accuracy: 0.9819
Epoch 7/10
23/23 [==============================] - 1s 46ms/step - loss: 0.2053 - accuracy: 0.9910
Epoch 8/10
23/23 [==============================] - 1s 46ms/step - loss: 0.1625 - accuracy: 0.9910
Epoch 9/10
23/23 [==============================] - 1s 46ms/step - loss: 0.1390 - accuracy: 0.9955
Epoch 10/10
23/23 [=====

/usr/local/lib/python3.10/dist-packages/ktrain/text/preprocessor.py:382: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn(


...

preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 16
	99percentile : 20


Is Multi-Label? False


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
22/22 [==============================] - 9s 46ms/step - loss: 2.8838 - accuracy: 0.0409
Epoch 2/10
22/22 [==============================] - 1s 46ms/step - loss: 2.6973 - accuracy: 0.3409
Epoch 3/10
22/22 [==============================] - 1s 46ms/step - loss: 1.9869 - accuracy: 0.8091
Epoch 4/10
22/22 [==============================] - 1s 46ms/step - loss: 1.0896 - accuracy: 0.9045
Epoch 5/10
22/22 [==============================] - 1s 46ms/step - loss: 0.5171 - accuracy: 0.9864
Epoch 6/10
22/22 [==============================] - 1s 46ms/step - loss: 0.2543 - accuracy: 0.9909
Epoch 7/10
22/22 [==============================] - 1s 46ms/step - loss: 0.1629 - accuracy: 0.9909
Epoch 8/10
22/22 [==============================] - 1s 46ms/step - loss: 0.1255 - accuracy: 0.9955
Epoch 9/10
22/22 [==============================] - 1s 46ms/step - loss: 0.1045 - accuracy: 1.0000
Epoch 10/10
22/22 [=====

/usr/local/lib/python3.10/dist-packages/ktrain/text/preprocessor.py:382: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn(


preprocessing train...
language: en
train sequence lengths:
	mean : 25
	95percentile : 87
	99percentile : 92


Is Multi-Label? False


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/10
17/17 [==============================] - 9s 46ms/step - loss: 2.8968 - accuracy: 0.0915
Epoch 2/10
17/17 [==============================] - 1s 46ms/step - loss: 2.7930 - accuracy: 0.2073
Epoch 3/10
17/17 [==============================] - 1s 47ms/step - loss: 2.3717 - accuracy: 0.5183
Epoch 4/10
17/17 [==============================] - 1s 46ms/step - loss: 1.5443 - accuracy: 0.8232
Epoch 5/10
17/17 [==============================] - 1s 46ms/step - loss: 0.8537 - accuracy: 0.9146
Epoch 6/10
17/17 [==============================] - 1s 46ms/step - loss: 0.4528 - accuracy: 0.9634
Epoch 7/10
17/17 [==============================] - 1s 46ms/step - loss: 0.2736 - accuracy: 0.9817
Epoch 8/10
17/17 [==============================] - 1s 46ms/step - loss: 0.1992 - accuracy: 0.9939
Epoch 9/10
17/17 [==============================] - 1s 46ms/step - loss: 0.1673 - accuracy: 0.9939
Epoch 10/10
17/17 [=====

In [31]:
full_df_result.head(10)

,dataset,model,test_accuracy,classifier,training_time
0,atis_full,NaN,0.9588,svm_tfidf,6.59
1,atis_full,NaN,0.9280,svm_glove,11.30
2,atis_full,NaN,0.9811,lstm_glove,57.61
3,atis_full,NaN,0.9588,bert_ktrain,6.81
4,trec_full,NaN,0.9599,svm_tfidf,81.58
5,trec_full,NaN,0.8177,svm_glove,38.93
6,trec_full,NaN,0.9724,lstm_glove,99.86
7,trec_full,NaN,0.9599,bert_ktrain,81.77


In [13]:
subsets_df_result.head(20)

,dataset,classifier,test_accuracy,model,subset_size,method,training_time
0,atis_5_subset_train,svm_tfidf,0.6055,None,5,None,0.04
1,atis_5_subset_train,svm_glove,0.5815,None,5,None,8.40
2,atis_5_subset_train,lstm_glove,0.5369,None,5,None,33.99
3,atis_5_subset_train,bert_ktrain,0.7925,None,5,None,17.93
4,trec_5_subset_train,svm_tfidf,0.4599,None,5,None,0.21
5,trec_5_subset_train,svm_glove,0.3177,None,5,None,8.60
6,trec_5_subset_train,lstm_glove,0.4254,None,5,None,35.30
7,trec_5_subset_train,bert_ktrain,0.5050,None,5,None,26.97


In [14]:
augmented_df_result.sort_values(by=['classifier', 'test_accuracy']).head(50)

,dataset,classifier,test_accuracy,model,subset_size,method,training_time
11,Llama3_8B_lambada_plus_ATIS_5_augmented_data_b...,bert_ktrain,0.6312,Llama3,5,Lambada+,23.40
19,Llama_8B_lambada_plus_instruct_ATIS_5_augmente...,bert_ktrain,0.6724,Llama3,5,Lambada+Instruct,21.86
15,Mistral_7B_lambada_plus_ATIS_5_augmented_data_...,bert_ktrain,0.7890,Mistral,5,Lambada+,23.84
3,Llama3_8B_ATIS_5_augmented_data_bert_ktrain_fi...,bert_ktrain,0.8782,Llama3,5,Lambada,24.28
7,Mistral_7B_ATIS_5_augmented_data_bert_ktrain_f...,bert_ktrain,0.8988,Mistral,5,Lambada,24.70
10,Llama3_8B_lambada_plus_ATIS_5_augmented_data_l...,lstm_glove,0.4425,Llama3,5,Lambada+,34.61
14,Mistral_7B_lambada_plus_ATIS_5_augmented_data_...,lstm_glove,0.4545,Mistral,5,Lambada+,34.26
18,Llama_8B_lambada_plus_instruct_ATIS_5_augmente...,lstm_glove,0.4648,Llama3,5,Lambada+Instruct,34.65
6,Mistral_7B_ATIS_5_augmented_data_lstm_glove_fi...,lstm_glove,0.4957,Mistral,5,Lambada,34.24
2,Llama3_8B_ATIS_5_augmented_data_lstm_glove_fil...,lstm_glove,0.6244,Llama3,5,Lambada,34.62


In [ ]:
df_result = pd.concat([full_df_result, subsets_df_result, augmented_df_result], ignore_index=True)
df_result.to_csv("final_results/test_results.csv")